In [ ]:
import pandas as pd
fact_publication = catalog.load('stg/fact_publication')

In [2]:
fact_publication

,title,doi,attrank,cc,downloads,icc,pr,ram,views,in_dspacedb,in_openaire,in_openalex
0,En la rehabilitación de la arquitectura en ado...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
1,El desarrollo regional a partir de la construc...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
2,Serotonin modulates proliferation of radial gl...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
3,¡Última llamada al pasajero Nansen!: reflexion...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
4,"Informe científico de investigador: Martino, P...",NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
11637,Geographical limits of the Southeastern distri...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
11638,Talbot effect by a photorefractive volume phas...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
11639,Respuesta de las poblaciones de rizobios de so...,NOT_SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
11640,Trabajadores del sexo y salud pública: interse...,10.18294/sc.2016.1205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
